**CLASSES**

In [9]:
class Reviews:
    def __init__(self, text ,score, sentiment):
        self.text = text
        self.score = score
        self.sentiment = sentiment

    def get_sentiment(self):
        if self.score <= 2:
            return self.sentiment
        elif self.score == 3:
            return self.sentiment
        else: 
            return self.sentiment

**LOAD DATA**

In [10]:
import json
import random

file = r"C:\Users\fathu\Documents\Sentiment Analysis Data\database.json"
reviews = []

with open(file) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Reviews(review['reviewText'], review['overall'] ,review['sentiment']))

negative = list(filter(lambda x: x.sentiment == 'NEGATIVE', reviews))
positive = list(filter(lambda x: x.sentiment == 'POSITIVE', reviews))
data = negative + positive
random.shuffle(data)

**DATA PREP**

In [11]:
import re
import string

X = [x.text for x in data]

df = pd.DataFrame({'text':X})

def text_cleaning(text):
    text = text.lower()
    text = re.sub('\[.*>\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

cleaned = lambda x: text_cleaning(x)
df['cleaned_text'] = pd.DataFrame(df['text'].apply(cleaned))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
from sklearn.model_selection import train_test_split

x = df['cleaned_text'].values.tolist()
y = [x.get_sentiment() for x in data]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


**MODELING**

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(X_train)

test_x_vectors = vectorizer.transform(X_test)

print(X_train[0])
print(train_x_vectors[0].toarray())

this is my experience with the game and the staff  read and judge for yourselvesfirst of all this game is fun  it is repetitive in its mechanics and game play  the tournaments where you can test yourself against other players characters and climb the leader board are the highlight to me  there do appear to be quite a few glitches and exploits and this appears to be a newer game so that isnt unexpectedive dealt with customer service twice since installthe first glitch i encountered was the result of an in app purchase  they got my money i didnt get my gemstones  i hit the customer service button under the gear icon menu after you launch the game and opened a ticket stating my problem and asking what information was needed from me  the first response relaying instructions was prompt  the second in which i received my merchandise took three days  at this point i was annoyed and thinking about writing up a scathing hate review when my email informs me that gameloft has responded  not only 

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

svc = svm.SVC()
pipe = Pipeline([("classifier", svc)])


params = [
    {"classifier": [svc],
    "classifier__kernel": ['linear', 'rbf'],
    "classifier__C": [1,4,8,16,32]
    },
    {"classifier": [DecisionTreeClassifier()],
    "classifier__criterion": ['gini', 'entropy'],
    "classifier__max_depth": [2,4,6,8,10,12]
    },
    {"classifier": [LogisticRegression()],
    "classifier__penalty": ['l1', 'l2'],
    "classifier__C": np.logspace(0, 4, 10)
    }
]

gridsearch = GridSearchCV(pipe, params, cv = 5, verbose=0, n_jobs=-1)
best_model = gridsearch.fit(train_x_vectors, y_train)

print(f'best models {best_model.best_estimator_}')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

C:\Users\fathu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.86084017 0.86412979 0.8455374  0.86749064 0.83495396 0.86749064
 0.8303771  0.86749064 0.82672977 0.86749064 0.63722719 0.65732181
 0.68092049 0.6889301  0.70180221 0.70616459 0.63272237 0.65188747
 0.6677616  0.68084942 0.69393664 0.70366175        nan 0.85869484
        nan 0.86248489        nan 0.86162698        nan 0.8554057
        nan 0.84610918        nan 0.84339183        nan 0.83938706
        nan 0.838958          nan 0.83774254        nan 0.83931563]
  warnings.warn(


best models Pipeline(steps=[('classifier', SVC(C=4))])


In [7]:
print("the mean accuracy:", best_model.score(test_x_vectors, y_test))
print(classification_report(y_test,best_model.best_estimator_.predict(test_x_vectors)))

the mean accuracy: 0.8715382048715382
              precision    recall  f1-score   support

    NEGATIVE       0.87      0.88      0.87      3038
    POSITIVE       0.87      0.87      0.87      2956

    accuracy                           0.87      5994
   macro avg       0.87      0.87      0.87      5994
weighted avg       0.87      0.87      0.87      5994



**SAVING MODEL**

In [8]:
import pickle

with open('sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(best_model, f)